# Schwellenwerte erstellen

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
THRESHOLD_BASE_FILE_PATH = ""  # input
THRESHOLD_BASE_SHEET_NAME = ""  # input
DEPARTMENT_INFORMATION_FILE_PATH = ""  # input
DEPARTMENT_INFORMATION_SHEET_NAME = ""  # input


In [ ]:
df_basis_threshold = pd.read_excel(THRESHOLD_BASE_FILE_PATH, sheet_name=THRESHOLD_BASE_SHEET_NAME)
df_basis_threshold = df_basis_threshold.astype("str")
# damit eval() funktioniert, musses en string si

df_department_information = pd.read_excel(DEPARTMENT_INFORMATION_FILE_PATH, sheet_name=DEPARTMENT_INFORMATION_SHEET_NAME) #Versucht mit dem Sheet alle Abteilungen reinzubringen
df_department_information = df_department_information.astype("str")
# damit .replace funktioniert, musses en string si


In [ ]:
departments = df_department_information.department.drop_duplicates()

dep_dict = {}
for department in departments:
    df_dep = df_department_information[df_department_information.department == department]
    dep_dict[department] = dict(zip(df_dep["variable"], df_dep["value"]))

In [ ]:
# sortiert nach inneren keys

for key in dep_dict.keys():
    new_d = {}
    for k in sorted(dep_dict[key], key=len, reverse=True):
        new_d[k] = dep_dict[key][k]
    dep_dict[key] = new_d

In [ ]:
print(dep_dict)

In [ ]:
df_thresholds = df_basis_threshold

for department in dep_dict:
    df_thresholds[f"{department}_threshold"] = df_basis_threshold["base_threshold"].replace(dep_dict[department], regex=True)

df_thresholds

In [ ]:
def isevaluatable(s):
    try:
        eval(s)
        return True
    except NameError:
        return False

In [ ]:
for department in dep_dict.keys():
    for index, row in df_thresholds.iterrows():
        s = str(row[f"{department}_threshold"])
        if isevaluatable(s) == True:
            df_thresholds.loc[index, f"{department}_threshold"] = eval(s)
        else:
            df_thresholds.loc[index, f"{department}_threshold"] = s

df_thresholds.replace("nan", np.NaN, inplace=True)

df_thresholds

# Daten-Tabelle erstellen

In [ ]:
# Daten zusammenführen
import glob

In [ ]:
DATA_FOLDER_PATH = ""  # input
MERGED_DATA_FILE_PATH = ""  # input


In [ ]:
path = DATA_FOLDER_PATH

all_files = glob.glob(path + "/*.xlsx")
file_list = []
for filename in all_files:
    df = pd.read_excel(filename, index_col=None, header=0)
    file_list.append(df)

df_collected_data = pd.concat(file_list, axis=0, ignore_index=True)
df_collected_data = df_collected_data.drop(columns=["q_59", "q_63"])


df_collected_data.to_csv(MERGED_DATA_FILE_PATH)


In [ ]:
# Überprüfen, ob Fragen, welche nur 1 Antwort haben sollten, wirklich nur 1 Antwort haben

one_value_list = ["q_17", "q_30", "q_31", "q_32", "q_33", "q_34", "q_35", "q_36", "q_37", "q_38", "q_39", "q_40", "q_41", "q_42", "q_43", "q_44", "q_45", "q_46", "q_47", "q_48", "q_49", "q_50", "q_51", "q_52", "q_54", "q_71", "q_72", "q_73", "q_74", "q_75", "q_76", "q_77", "q_83", "q_84", "q_85", "q_86", "q_88", "q_89", "q_90", "q_91", "q_92", "q_93", "q_95", "q_97", "q_98", "q_99", "q_100", "q_102", "q_202"]

for item in one_value_list:
    if item in df_collected_data.columns:
        assert df_collected_data[item].count() == 1, f"{item}: zu viele Antworten"

In [ ]:
# Überprüfen, ob alle Spalten mind. 1 Antwort enthalten

for col in df_collected_data:
    answers_count = df_collected_data[col].count()
    assert answers_count > 0, f"keine Antwort für Spalte {col}"

## Ist-Werte Berechnen

In [ ]:
MAPPING_FILE_PATH = ""  # input
MAPPING_FILE_SHEET_NAME = ""  # input

THRESHOLDS_FILE_PATH = ""  # output

In [ ]:
df_data = pd.read_csv(MERGED_DATA_FILE_PATH)
df_mapping = pd.read_excel(MAPPING_FILE_PATH, sheet_name=MAPPING_FILE_SHEET_NAME)

df_mapping

# TO DO: hier vielleicht check einbauen, dass keine "operation" ein "none" ist

In [ ]:
## Vorlage ##

# for sub_indicator in df_mapping["sub_id"].drop_duplicates():
#     q_ids = df_mapping.loc[df_mapping["sub_id"] == sub_indicator, "q_id"].dropna()
#     q_ids = q_ids[q_ids.isin(df_data.columns)]
#     if len(q_ids) == 0:
#         continue
#     ops = df_mapping.loc[df_mapping["sub_id"] == sub_indicator, "operation"].iloc[0] # der erste operator; z.B. wenn es mehrere q_id's hat, dann einfach der erste operator
#     df = df_data.loc[:, q_ids]
#     if ops == "mean":
#         df = df.applymap(lambda x: x).mean().mean()
#         print(sub_indicator, df)
#     elif ops == "sum":
#         df = df.applymap(lambda x: x).sum().sum()
#     else:
#         raise ValueError(f"Unknown operation {ops}")

In [ ]:
df_thresholds_values = df_thresholds
values = {}

for department in dep_dict.keys():
    for sub_indicator in df_mapping["sub_id"].drop_duplicates():
        q_ids = df_mapping.loc[df_mapping["sub_id"] == sub_indicator, "q_id"].dropna()
        q_ids = q_ids[q_ids.isin(df_data.columns)]
        if len(q_ids) == 0:
            continue
        ops = df_mapping.loc[df_mapping["sub_id"] == sub_indicator, "operation"].iloc[0] # der erste operator; z.B. wenn es mehrere q_id's hat, dann einfach der erste operator
        df = df_data.loc[df_data['q_500'] == department, q_ids]
        if ops == "mean":
            df = df.applymap(lambda x: x).mean().mean()
            values[sub_indicator] = df
        elif ops == "sum":
            df = df.applymap(lambda x: x).sum().sum()
            values[sub_indicator] = df
        elif ops == 'none':
            num_na = int(df.notna().sum())
            if num_na == 0:
               values[sub_indicator] = np.nan
            elif num_na == 1:
               values[sub_indicator] = df[df.notna()]
            else:
                raise ValueError(f"There are more than 1 non-empty rows for column {q_ids}")
        else:
            raise ValueError(f"Unknown operation {ops}, {type(ops)}")
    df_thresholds_values[f"{department}_value"] = df_thresholds_values["sub_id"].apply(lambda x: values.get(x))

In [ ]:
df_thresholds.to_csv(THRESHOLDS_FILE_PATH)

## Schwellenwert mit Ist-Wert vergleichen

In [ ]:
PERFORMANCE_FILE_PATH = ""  # outupt

In [ ]:
df_performance = df_thresholds_values

for department in dep_dict.keys():
    df_performance[f"{department}_performance"] = ""

    for index, row in df_performance.iterrows():
        if row[f"{department}_threshold"] == np.nan:
            continue
        else:
            if row["operator"] == ">":
                if row[f"{department}_value"] > row[f"{department}_threshold"]:
                    df_performance[f"{department}_performance"].loc[index] = True
                    # print(f"{department}:> true")
                else:
                    df_performance[f"{department}_performance"].loc[index] = False
                    # print(f"{department}:> false")
            elif row["operator"] == ">=":
                if row[f"{department}_value"] >= row[f"{department}_threshold"]:
                    df_performance[f"{department}_performance"].loc[index] = True
                    # print(f"{department}:>= true")
                else:
                    df_performance[f"{department}_performance"].loc[index] = False
                    # print(f"{department}:>= false")
            elif row["operator"] == "<":
                if row[f"{department}_value"] < row[f"{department}_threshold"]:
                    df_performance[f"{department}_performance"].loc[index] = True
                    # print(f"{department}:< true")
                else:
                    df_performance[f"{department}_performance"].loc[index] = False
                    # print(f"{department}:< false")
            elif row["operator"] == "<=":
                if row[f"{department}_value"] <= row[f"{department}_threshold"]:
                    df_performance[f"{department}_performance"].loc[index] = True
                    # print(f"{department}:<= true")
                else:
                    df_performance[f"{department}_performance"].loc[index] = False
                    # print(f"{department}:<= false")
            elif row["operator"] == "=":
                if row[f"{department}_value"] == row[f"{department}_threshold"]:
                    df_performance[f"{department}_performance"].loc[index] = True
                    # print(f"{department}:== true")
                else:
                    df_performance[f"{department}_performance"].loc[index] = False
                    # print(f"{department}:== false")
            elif row["operator"] == " =":
                if row[f"{department}_value"] == row[f"{department}_threshold"]:
                    df_performance[f"{department}_performance"].loc[index] = True
                    # print(f"{department}:== true")
                else:
                    df_performance[f"{department}_performance"].loc[index] = False
                    # print(f"{department}: == false")
            elif row["operator"] == "status quo":
                df_performance[f"{department}_performance"].loc[index] = np.nan
            elif row["operator"] == "nan":
                df_performance[f"{department}_performance"].loc[index] = np.nan
            else:
                print(f"operator is not assigned: row {index}")

df_performance.to_csv(PERFORMANCE_FILE_PATH)
